In [1]:
import pandas as pd
import numpy as np


In [2]:
outcomes = ['B2', 'B3', 'rem', 'surg', 'cd']
res_dfs = {}
for outcome in outcomes:
    res_dfs[outcome] = pd.read_csv(f'results/c/{outcome}/results_deseq.csv', index_col=0)

In [12]:
res_dfs['cd'].head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,ensembl
gene,,,,,,,
AQP9,212.314584,6.479299,0.586156,11.053884,2.099320e-28,6.565204e-24,NaN
CXCL5,602.253882,5.797342,0.528648,10.966360,5.546065e-28,8.672105e-24,NaN
CXCL8,352.505795,4.940754,0.461909,10.696376,1.058371e-26,1.103281e-22,NaN
S100A9,124.661701,4.867321,0.485405,10.027341,1.155869e-23,9.036873e-20,NaN
S100A8,105.702119,5.831763,0.588773,9.904950,3.961678e-23,2.477871e-19,NaN


In [3]:
deg_counts = pd.DataFrame(index=outcomes, columns=['count'])
for outcome in outcomes:
    deg_counts.loc[outcome, 'count'] = len(res_dfs[outcome].loc[res_dfs[outcome]['padj'] < 0.05])

In [4]:
deg_counts.head()

,count
B2,284
B3,2
rem,0
surg,4
cd,10973


In [5]:
deg_counts.to_csv('results/c/deg_counts.csv')

In [7]:
ens2gene = pd.read_csv('hg38_ensembl2gene.txt', sep='\t')
ens2gene.head()

,ensembl,biotype,gene_id,chr,start,stop
0,ENSG00000223972.5,transcribed_unprocessed_pseudogene,DDX11L1,chr1,11869,14409
1,ENSG00000227232.5,unprocessed_pseudogene,WASH7P,chr1,14404,29570
2,ENSG00000278267.1,miRNA,MIR6859-1,chr1,17369,17436
3,ENSG00000243485.5,lncRNA,MIR1302-2HG,chr1,29554,31109
4,ENSG00000284332.1,miRNA,MIR1302-2,chr1,30366,30503


In [29]:
#remove version number from ensembl id
ens2gene['ensembl'] = ens2gene['ensembl'].str.split('.').str[0]
#remove empty spaces from gene id
ens2gene['gene_id'] = ens2gene['gene_id'].str.strip()
#drop duplicated ensembl ids from ens2gene
ens2gene.drop_duplicates(subset='ensembl', keep='first', inplace=True)

In [30]:
for outcome in outcomes:
    res_dfs[outcome]['ensembl'] = np.NaN
    res_dfs[outcome].reset_index(inplace=True)

In [31]:
#fill ensembl with ens2gene['ensembl'], except for rows that start with 'ENSG'
for outcome in outcomes:
    for i in range(len(res_dfs[outcome])):
        if res_dfs[outcome]['gene'][i].startswith('ENSG'):
            res_dfs[outcome]['ensembl'][i] = res_dfs[outcome]['gene'][i]
        else:
            try:
                res_dfs[outcome]['ensembl'][i] = ens2gene.loc[ens2gene['gene_id'] == res_dfs[outcome]['gene'][i], 'ensembl'].values[0]
            except IndexError:
                res_dfs[outcome]['ensembl'][i] = np.NaN

/tmp/ipykernel_2451037/3619566027.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_dfs[outcome]['ensembl'][i] = ens2gene.loc[ens2gene['gene_id'] == res_dfs[outcome]['gene'][i], 'ensembl'].values[0]
/home/kchen/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [36]:
res_dfs['cd'].head()

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,biotype
ensembl,,,,,,,,
ENSG00000103569,AQP9,212.314584,6.479299,0.586156,11.053884,2.099320e-28,6.565204e-24,protein_coding
ENSG00000163735,CXCL5,602.253882,5.797342,0.528648,10.966360,5.546065e-28,8.672105e-24,protein_coding
ENSG00000169429,CXCL8,352.505795,4.940754,0.461909,10.696376,1.058371e-26,1.103281e-22,protein_coding
ENSG00000163220,S100A9,124.661701,4.867321,0.485405,10.027341,1.155869e-23,9.036873e-20,protein_coding
ENSG00000143546,S100A8,105.702119,5.831763,0.588773,9.904950,3.961678e-23,2.477871e-19,protein_coding


In [33]:
for outcome in outcomes:
    res_dfs[outcome].reset_index(inplace=True, drop=True)
    res_dfs[outcome].drop(columns=['index'], inplace=True)

In [35]:
for outcome in outcomes:
    res_dfs[outcome].set_index('ensembl', inplace=True)
    res_dfs[outcome]['biotype'] = np.NaN
    res_dfs[outcome].update(ens2gene.set_index('ensembl'))

In [37]:
for outcome in outcomes:
    res_dfs[outcome].drop(columns=['lfcSE', 'stat'], inplace=True)

In [38]:
for outcome in outcomes:
    res_dfs[outcome].to_csv(f'results/c/{outcome}/results_deseq_processed.csv')

In [39]:
#same thing for ileum results

outcomes = ['B2', 'B3', 'rem', 'surg', 'cd']
res_dfs = {}
for outcome in outcomes:
    res_dfs[outcome] = pd.read_csv(f'results/i/{outcome}/results_deseq.csv', index_col=0)
res_dfs['cd'].head()


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene,,,,,,
HCAR3,340.536587,6.085196,0.475881,12.787222,1.932581e-37,6.088791e-33
AQP9,272.762218,6.091092,0.510950,11.921118,9.186694e-33,1.447180e-28
CXCL8,326.328961,4.998463,0.422670,11.825924,2.867289e-32,2.475974e-28
MUC1,740.333249,2.762535,0.233753,11.818200,3.143495e-32,2.475974e-28
MMP3,600.968378,6.544629,0.563434,11.615606,3.433449e-31,2.163485e-27


In [40]:
deg_counts = pd.DataFrame(index=outcomes, columns=['count'])
for outcome in outcomes:
    deg_counts.loc[outcome, 'count'] = len(res_dfs[outcome].loc[res_dfs[outcome]['padj'] < 0.05])
deg_counts.head()


,count
B2,0
B3,1
rem,0
surg,1
cd,8799


In [41]:
deg_counts.to_csv('results/i/deg_counts.csv')


In [42]:
ens2gene = pd.read_csv('hg38_ensembl2gene.txt', sep='\t')
#remove version number from ensembl id
ens2gene['ensembl'] = ens2gene['ensembl'].str.split('.').str[0]
#remove empty spaces from gene id
ens2gene['gene_id'] = ens2gene['gene_id'].str.strip()
#drop duplicated ensembl ids from ens2gene
ens2gene.drop_duplicates(subset='ensembl', keep='first', inplace=True)
for outcome in outcomes:
    res_dfs[outcome]['ensembl'] = np.NaN
    res_dfs[outcome].reset_index(inplace=True)
#fill ensembl with ens2gene['ensembl'], except for rows that start with 'ENSG'
for outcome in outcomes:
    for i in range(len(res_dfs[outcome])):
        if res_dfs[outcome]['gene'][i].startswith('ENSG'):
            res_dfs[outcome]['ensembl'][i] = res_dfs[outcome]['gene'][i]
        else:
            try:
                res_dfs[outcome]['ensembl'][i] = ens2gene.loc[ens2gene['gene_id'] == res_dfs[outcome]['gene'][i], 'ensembl'].values[0]
            except IndexError:
                res_dfs[outcome]['ensembl'][i] = np.NaN
res_dfs['cd'].head()


/tmp/ipykernel_2451037/433113150.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_dfs[outcome]['ensembl'][i] = ens2gene.loc[ens2gene['gene_id'] == res_dfs[outcome]['gene'][i], 'ensembl'].values[0]
/home/kchen/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/tmp/ipykernel_2451037/433113150.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

,gene,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,ensembl
0,HCAR3,340.536587,6.085196,0.475881,12.787222,1.932581e-37,6.088791e-33,ENSG00000255398
1,AQP9,272.762218,6.091092,0.510950,11.921118,9.186694e-33,1.447180e-28,ENSG00000103569
2,CXCL8,326.328961,4.998463,0.422670,11.825924,2.867289e-32,2.475974e-28,ENSG00000169429
3,MUC1,740.333249,2.762535,0.233753,11.818200,3.143495e-32,2.475974e-28,ENSG00000185499
4,MMP3,600.968378,6.544629,0.563434,11.615606,3.433449e-31,2.163485e-27,ENSG00000149968


In [44]:
for outcome in outcomes:
    res_dfs[outcome].reset_index(inplace=True, drop=True)
for outcome in outcomes:
    res_dfs[outcome].set_index('ensembl', inplace=True)
    res_dfs[outcome]['biotype'] = np.NaN
    res_dfs[outcome].update(ens2gene.set_index('ensembl'))
    res_dfs[outcome].drop(columns=['lfcSE', 'stat'], inplace=True)    
for outcome in outcomes:
    res_dfs[outcome].to_csv(f'results/i/{outcome}/results_deseq_processed.csv')

In [45]:
res_dfs['cd']['biotype'].value_counts()

 protein_coding                        16869
 lncRNA                                 9807
 processed_pseudogene                   1340
 TEC                                     662
 transcribed_unprocessed_pseudogene      588
 misc_RNA                                556
 unprocessed_pseudogene                  287
 snRNA                                   282
 transcribed_processed_pseudogene        249
 snoRNA                                  227
 miRNA                                   187
 IG_V_gene                               109
 transcribed_unitary_pseudogene           94
 TR_J_gene                                61
 TR_V_gene                                37
 Mt_tRNA                                  22
 scaRNA                                   19
 unitary_pseudogene                       16
 polymorphic_pseudogene                   16
 IG_J_gene                                14
 IG_C_gene                                14
 IG_V_pseudogene                          11
 pseudogen